[Advent of Code - Day 24](https://adventofcode.com/2022/day/24)

# Import input

In [1]:
import sys
from math import lcm
from heapq import heappush, heappop

sys.path.insert(0, "../")
from utils import puzzle

P = puzzle.Puzzle(year="2022", day="24")
INPUT = P.load_input()

In [2]:
INPUT[:5]

['#.########################################################################################################################',
 '#>v>^>^<<^>v^>vv<.><>v^>vvv><.>^^.<.^>^vv>v^v>.>>>.v^>>^<v<v<.<<>v^v<^>^<<v>^<<v>><>^v<.^v^^v<v>>>^<.v<^^v>^^<>><v^v^<.v.#',
 '#.>.^<.>^<v<.v>v^<^>>^v^>v>v<.v>v^v.>^><>v<>.^v><>.<v^>.v<<^..>^<<^<<>vv^<v<><v<^^>>v^><.>v<v<^>^^^^>..<<><>>^>^^<.>>v...#',
 '#<<.><<<v^vv^<>>..<<>>>^<.<v^..vv><^vv^^<^<>>v<^>.<>^>v<>^>v><v^>v^>v><>>v^>vv>v^v>.>^v<^<^><^vv<^v>v^^v^.^<>v.^<>^.>v>v<#',
 '#<^v^^><^^<^<v^^<^^<>.<^<v>v..<v^^vvv<>><v^.<>>><>v>^v>v<v.v.vv<vv<>><>><^^<vv>.>>.<>>^>v>^v.>.><<<<>^>^>v>v>^><^^v<^vv<>#']

# Solulu

Inspired by [silentw0lf's solution](https://github.com/silentw0lf/advent_of_code_2022/blob/main/24/solve.py#L35)

In [3]:
height, width = len(INPUT), len(INPUT[0])
start = (0, INPUT[0].index("."))
end = (height - 1, INPUT[height - 1].index("."))

In [4]:
def solve(start: tuple, end: tuple, start_time: int = 0) -> int:
    time = start_time
    visited = set()
    queue = []
    heappush(queue, (time, start))

    while True:
        time, (row, col) = heappop(queue)
        state = time % blizzard_states

        if (row, col, state) not in visited:
            visited.add((row, col, state))
            if (row, col) == end:
                return time - start_time

            moves = [
                (row - 1, col),
                (row + 1, col),
                (row, col - 1),
                (row, col + 1),
                (row, col),
            ]
            for move in moves:
                r, c = move
                if is_valid(*move, state):
                    heappush(queue, (time + 1, move))

In [5]:
def is_valid(row, col, state) -> bool:
    global height, width

    if row in range(height) and col in range(width):
        if INPUT[row][col] == "#" or (row, col) in blizzard_coords[state]:
            return False
        else:
            return True
    else:
        return False

In [6]:
def update(blizzards: dict) -> dict:
    global height, width
    new_blizzards = {coord: [] for coord in blizzards.keys()}

    for (row, col), blizz in blizzards.items():
        blizz_copy = blizz.copy()

        while blizz_copy:
            bl = blizz_copy.pop(0)
            if bl == "^":
                new_row = height - 2 if row == 1 else row - 1
                new_col = col

            elif bl == "v":
                new_row = 1 if row == height - 2 else row + 1
                new_col = col

            elif bl == "<":
                new_row = row
                new_col = width - 2 if col == 1 else col - 1

            elif bl == ">":
                new_row = row
                new_col = 1 if col == width - 2 else col + 1

            new_blizzards[(new_row, new_col)].append(bl)

    return new_blizzards

In [7]:
blizzards = dict()

for irow in range(height):
    for icol in range(width):
        if INPUT[irow][icol] == "#":
            continue
        blizzards[(irow, icol)] = []

        if INPUT[irow][icol] in ["^", "v", "<", ">"]:
            blizzards[(irow, icol)].append(INPUT[irow][icol])

In [8]:
blizzard_states = lcm(height - 2, width - 2)
blizzard_coords = {}

for state in range(0, blizzard_states):
    blizzards = update(blizzards)
    blizzard_coords[state] = set([coord for coord, blizz in blizzards.items() if blizz])

## Part 1

In [9]:
solve(start=start, end=end, start_time=0)

257

## Part 2

In [10]:
time = solve(start=start, end=end, start_time=0)
time += solve(start=end, end=start, start_time=time)
time += solve(start=start, end=end, start_time=time)

time

828